In [1]:
# Unpack the pickle file (Mainly a Colab thing)
!unzip pickle.zip

Archive:  pickle.zip
   creating: pickle/
  inflating: pickle/.DS_Store        
  inflating: __MACOSX/pickle/._.DS_Store  
   creating: pickle/DHFR/
   creating: pickle/CRT/
   creating: pickle/DHPS/
  inflating: pickle/DHFR/trimmed_data_GB4.pkl  
  inflating: pickle/DHFR/trimmed_data_DD2.pkl  
  inflating: pickle/DHFR/trimmed_data_KH2.pkl  
  inflating: pickle/DHFR/trimmed_data_HB3.pkl  
  inflating: pickle/DHFR/trimmed_data_KH1.pkl  
  inflating: pickle/DHFR/trimmed_data_mix1.pkl  
  inflating: pickle/DHFR/trimmed_data_mix2.pkl  
  inflating: pickle/DHFR/trimmed_data_3D7.pkl  
  inflating: pickle/DHFR/trimmed_data_7G8.pkl  
  inflating: pickle/CRT/trimmed_data_DD2.pkl  
  inflating: pickle/CRT/trimmed_data_KH2.pkl  
  inflating: pickle/CRT/trimmed_data_KH1.pkl  
  inflating: pickle/CRT/trimmed_data_mix1.pkl  
  inflating: pickle/CRT/trimmed_data_mix2.pkl  
  inflating: pickle/CRT/trimmed_data_3D7.pkl  
  inflating: pickle/CRT/trimmed_data_7G8.pkl  
  inflating: pickle/DHPS/trimmed_da

In [2]:
import HaploReader as hp
import VariantCalling as vc
import numpy as np

# User-defined parameteres

In [3]:
run_sequence = "CRT"
nb_epoch = 100
batch_size = 128
nb_filters = 48
nb_conv = 5
nb_layer_conv = 3
nb_pool = 3
nb_alignment = 2500
nb_channel = 5
nb_coverage = 100
nb_node = 128
nb_layer_ff = 2
train_loop = 0

# This is the library for the corresponding parameters for each sequence

In [4]:
if run_sequence == "CRT":
    img_row, img_col = nb_coverage + 1, 178
    clone_names = ["3D7","7G8","DD2"]
    nb_mutations = len(clone_names)
    ref_file_name = "clones.txt"
elif run_sequence == "DHPS":
    img_row, img_col = nb_coverage + 1, 642
    clone_names = ["3D7","DD2","HB3"]
    nb_mutations = len(clone_names)
    ref_file_name = "dhps_clones.txt"
elif run_sequence == "DHFR":
    img_row, img_col = nb_coverage + 1, 491
    clone_names = ["3D7","7G8","DD2","HB3"]
    nb_mutations = len(clone_names)
    ref_file_name = "dhfr_clones.txt"
#  Concatenate this section in the future for other genome sequences

# Synthesize the pile-up data

In [7]:
dg = vc.VariantCallingData(file_name=ref_file_name,gen_mode=2,pkl_sequence=run_sequence,pkl_clones=clone_names)
alignments_raw, prob_lists = dg.simulate_clones(nb_alignment,nb_coverage,0.01,0.01,verbose=1)
prob_lists = (np.array(prob_lists)*100).tolist()
alignments = [dg._array_constituent_bp(i,101,True) for i in alignments_raw]
alignments = dg.char_to_int(alignments)

Done, Number of alignments: 2500


# Create a model-builder object based on the user-defined parameters

In [8]:

ref_train = hp.HaploReaderTFTrain(alignments, prob_lists, nb_epoch, batch_size, nb_filters, nb_conv, nb_layer_conv, nb_pool, nb_node, nb_layer_ff, train_loop)

In [9]:
print(np.shape(alignments)[0])
print(np.shape(ref_train.prob_lists)[1])

2500
3


# Perform train-val split followed by model train
The model training shall be run without issue

In [10]:
ref_train.train_val_split()
ref_train.model_train()

Epoch 1/100
16/16 [==============================] - 14s 101ms/step - loss: 109.8843 - mse: 312.7353 - val_loss: 109.8214 - val_mse: 314.7608
Epoch 2/100
16/16 [==============================] - 1s 46ms/step - loss: 109.8733 - mse: 312.5061 - val_loss: 109.8496 - val_mse: 315.3971
Epoch 3/100
16/16 [==============================] - 1s 44ms/step - loss: 109.8123 - mse: 311.1409 - val_loss: 109.7860 - val_mse: 313.9890
Epoch 4/100
16/16 [==============================] - 1s 46ms/step - loss: 109.7011 - mse: 308.6612 - val_loss: 109.3676 - val_mse: 304.6934
Epoch 5/100
16/16 [==============================] - 1s 44ms/step - loss: 108.2359 - mse: 275.8175 - val_loss: 104.8807 - val_mse: 203.2970
Epoch 6/100
16/16 [==============================] - 1s 45ms/step - loss: 103.5111 - mse: 172.7669 - val_loss: 100.7028 - val_mse: 115.6345
Epoch 7/100
16/16 [==============================] - 1s 61ms/step - loss: 100.1940 - mse: 105.7276 - val_loss: 97.8441 - val_mse: 62.5621
Epoch 8/100
16/16 [=

In [11]:
ref_train.model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 21, 36, 48)        6048      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 7, 12, 48)        0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 7, 12, 48)         0         
                                                                 
 activation (Activation)     (None, 7, 12, 48)         0         
                                                                 
 conv2d_1 (Conv2D)           (None, 2, 3, 48)          57648     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 1, 1, 48)         0         
 2D)                                                    

# Note:
The advantage of this setup is the object can be modified to take another alingment pile-up data without the need to repeat the code for the model definition